In [ ]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))
sc <- sparkR.init(master = "local[*]")
sparkR.session(master = "local[*]")

# 使用 RDD 法
[SparkR RDD API](http://amplab-extras.github.io/SparkR-pkg/rdocs/1.2/)

**自Spark ver. 1.4後，SparkR不能直接使用RDD相關的操作，需加上`SparkR:::`才能使用。**

將文字檔案內容讀到RDD裡。

In [ ]:
rdd <- SparkR:::textFile(sc, "/opt/spark/README.md")

將每一行文字切割成多個單字。

In [ ]:
words <- SparkR:::flatMap(rdd, function(line) {
  strsplit(line, " ")[[1]]
})

針對每一個單字，產生`(word, 1)`這樣形式的鍵值組。

In [ ]:
wordCount <- SparkR:::lapply(words, function(word) {
  list(word, 1)
})

將有相同單字的所有1累加起來。

In [ ]:
counts <- SparkR:::reduceByKey(wordCount, "+", 2)

將RDD轉成R的list

In [ ]:
op <- SparkR:::collectRDD(counts)

In [ ]:
op

# 使用 DataFrame 法

[SparkR DataFrame API](http://spark.apache.org/docs/latest/api/R/index.html)

目前SparkR主要是以DataFrame做為操作方法，後續所有範例都以DataFrame API為主。

將檔案讀到DataFrame

In [ ]:
df <- read.text('/opt/spark/README.md')
head(df)

分割出一行文字內所有的單字，構成另一個DataFrame

In [ ]:
word <-unlist(strsplit(collect(select(df, "value"))[,], " "))
word_df <- as.DataFrame(data.frame(word))
head(word_df)

將`word_df`裡相同word的單字群聚在一起後，再計算有多少個。

In [ ]:
word_counts <- summarize(groupBy(word_df, word_df$word), count = n(word_df$word))
head(word_counts)

依`count`的欄位排序後顯示。

In [ ]:
head(arrange(word_counts, desc(word_counts$count)))